In [153]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [188]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd

sns.set_theme(style="white")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [189]:
df = pd.read_csv("../data/processed/df_procesada.csv")

In [199]:
# Eliminamos las filas que no tienen fecha de evaluación
df_limpia = df.dropna(subset="fecha_1_evaluacion").copy()
df_limpia = df_limpia.query("anio_1_evaluacion >= 2000")

df_limpia["region"] = df_limpia["region"].str.replace(", Chile", "")

df_limpia["fecha_1_evaluacion"] = pd.to_datetime(df_limpia["fecha_1_evaluacion"])
df_limpia["f_nac"] = pd.to_datetime(df_limpia["f_nac"])
df_limpia["edad_1_evaluacion_calc"] = (
    df_limpia["fecha_1_evaluacion"] - df_limpia["f_nac"]
).dt.days / 365

In [206]:
# Obtiene la cantidad de pacientes anuales
cantidad_de_pacientes = df_limpia.groupby("anio_1_evaluacion")["rut"].nunique()

# Obtiene la distribucion de la prevision a lo largo de los anios
prevision = df_limpia.groupby("anio_1_evaluacion")["prevision"].value_counts().unstack()
prevision.loc["Total", :] =  prevision.sum()

# Obtiene la distribucion de la 1era edad de consulta
edad_primera_consulta = df_limpia.groupby("anio_1_evaluacion")["edad_1_evaluacion_calc"].describe()

# Obtiene el desglose por region
desglose_por_region = df_limpia["region"].value_counts()

In [209]:
desglose_por_region.to_frame().join(map_df.iloc[:, :7].set_index("Region"))

,count,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_
region,,,,,,,
Región Metropolitana de Santiago,1452,1092.0,7.0,13.0,15392.030737,2.225204e+10,1.064253e+06
Región Del Bíobío,230,NaN,NaN,NaN,NaN,NaN,NaN
Región Del Maule,227,NaN,NaN,NaN,NaN,NaN,NaN
Región De Coquimbo,219,NaN,NaN,NaN,NaN,NaN,NaN
Región del Libertador General Bernardo O'Higgins,178,NaN,NaN,NaN,NaN,NaN,NaN
Región De Valparaíso,161,NaN,NaN,NaN,NaN,NaN,NaN
Región De Los Lagos,151,NaN,NaN,NaN,NaN,NaN,NaN
Región De La Araucanía,82,NaN,NaN,NaN,NaN,NaN,NaN
Región De Antofagasta,61,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
map_df = gpd.read_file("../data/raw/mapas/Regional.shp")

In [ ]:
map["Region"] = 

In [203]:
map_df.iloc[:, :7]

,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,Region
0,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,Región de Arica y Parinacota
1,1085,2,1,42284.567484,4.830637e+10,1.213713e+06,Región de Tarapacá
2,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,Región de Antofagasta
3,1087,15,12,133053.142345,3.581316e+11,9.049830e+07,Región de Magallanes y Antártica Chilena
4,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,Región de Aysén del Gral.Ibañez del Campo
5,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,Región de Atacama
6,1090,5,4,40575.901682,5.498082e+10,2.065933e+06,Región de Coquimbo
7,1091,6,5,16322.970393,2.301475e+10,1.679609e+06,Región de Valparaíso
8,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,Región Metropolitana de Santiago
9,1093,13,10,48408.371357,8.771834e+10,7.874158e+06,Región de Los Lagos


In [ ]:
CAMBIOS_REGIONES = {
    "Región Del Bíobío": "Región del Bío-Bío",
    "Región Del Maule": "Región del Maule",
    "Región De Coquimbo": "Región de Coquimbo",
    "Región del Libertador General Bernardo O'Higgins": "Región del Libertador Bernardo O'Higgins",
    "Región De Valparaíso": "Región de Valparaíso",
    "Región De Los Lagos": "Región de Los Lagos",
    "Región De La Araucanía": "Región de La Araucanía",
}